In [232]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings

In [233]:
myData=pd.read_csv('imputedData.csv')
myData.head()

#One hot encoding of categorical variables

#Create list with features to be dummified cols.
nonum_feats_names = ['Month','OperatingSystems','Browser','Region','VisitorType','Weekend']


dataModel = pd.concat([myData[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']],
                       pd.get_dummies(myData[nonum_feats_names].astype('category')),myData['Revenue']],axis=1)
                      
dataModel.info()


#Label Encoding of revenue

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
myData['Revenue'] = le.fit_transform(myData['Revenue'])
myData['Revenue'].value_counts()

myData['Revenue'].head()

# getting dependent and independent variables

x = dataModel
# removing the target column revenue from x
x = x.drop(['Revenue'], axis = 1)

y = myData['Revenue']

# checking the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 57 columns):
Administrative                   12330 non-null int64
Administrative_Duration          12330 non-null float64
Informational                    12330 non-null int64
Informational_Duration           12330 non-null float64
ProductRelated                   12330 non-null int64
ProductRelated_Duration          12330 non-null float64
BounceRates                      12330 non-null float64
ExitRates                        12330 non-null float64
PageValues                       12330 non-null float64
TrafficType                      12330 non-null int64
SpecialDay                       12330 non-null float64
Month_Aug                        12330 non-null uint8
Month_Dec                        12330 non-null uint8
Month_Feb                        12330 non-null uint8
Month_Jul                        12330 non-null uint8
Month_June                       12330 non-null uint8
Month_Mar    

In [234]:
# splitting the data

from sklearn.model_selection import train_test_split

X_Train, X_Test, y_Train, y_Test = train_test_split(x, y, test_size = 0.3, random_state = 42,shuffle=True,stratify=y)

# checking the shapes

print("Shape of x_train :", X_Train.shape)
print("Shape of y_train :", y_Train.shape)
print("Shape of x_test :", X_Test.shape)
print("Shape of y_test :", y_Test.shape)

Shape of x_train : (8631, 56)
Shape of y_train : (8631,)
Shape of x_test : (3699, 56)
Shape of y_test : (3699,)


In [235]:
pd.DataFrame(y_Train,columns=['Revenue']).Revenue.value_counts(normalize=True)

0    0.845209
1    0.154791
Name: Revenue, dtype: float64

In [236]:
#SCALING WITH STANDARD Z SCORE SCALER
#Scaling the data first fitting it and transforming the training set
#to later apply the fit to transform the test set.
from sklearn.preprocessing import StandardScaler

num_cols_names = ['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']
#Instantiate Satandard Scaler
scaler = StandardScaler()
#Fit transform the numerical features in the training dataset to a new dataframe
scaled_numfeats_train = pd.DataFrame(scaler.fit_transform(X_Train[num_cols_names]), 
                                     columns=num_cols_names, index= X_Train.index)
#Integrate scaled values to the training set
for col in num_cols_names:
    X_Train[col] = scaled_numfeats_train[col]
    
    
#Transform the numerical features inthe training dataset to a new dataframe
scaled_numfeats_test = pd.DataFrame(scaler.transform(X_Test[num_cols_names]),
                                    columns=num_cols_names, index= X_Test.index)
#Integrate scaled values to the test set
for col in num_cols_names:
    X_Test[col] = scaled_numfeats_test[col]

C:\Users\neera\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\neera\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [237]:
X_Train.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,TrafficType,...,Region_5,Region_6,Region_7,Region_8,Region_9,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
7460,0.508002,-0.200488,-0.3947,-0.244336,-0.399509,-0.483009,-0.454346,-0.668356,-0.320111,0.223142,...,0,0,0,0,0,1,0,0,1,0
4687,-0.397089,-0.229562,-0.3947,-0.244336,-0.283724,-0.388159,-0.454346,-0.668356,-0.320111,-0.023995,...,0,0,0,0,0,0,0,1,1,0
790,-0.698787,-0.456792,-0.3947,-0.244336,-0.422666,-0.538272,-0.454346,-0.251598,-0.320111,-0.271132,...,0,0,0,0,0,0,0,1,1,0
9057,1.111396,-0.102539,-0.3947,-0.244336,0.341513,-0.040404,-0.308034,-0.544755,-0.320111,0.964551,...,0,0,0,0,0,0,0,1,1,0
225,-0.698787,-0.456792,-0.3947,-0.244336,-0.677393,-0.669486,-0.454346,1.173716,-0.320111,1.705961,...,0,0,0,0,0,0,0,1,0,1


In [238]:
X_Train.drop(labels=['Browser_1','BounceRates','ProductRelated','VisitorType_Returning_Visitor'], 
                 axis=1, inplace=True)
X_Test.drop(labels=['Browser_1','BounceRates','ProductRelated','VisitorType_Returning_Visitor'], 
                axis=1, inplace=True)

C:\Users\neera\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [239]:
X_Train = X_Train.drop(['Weekend_True','Region_2','Region_1','Month_Jul','Month_Feb','Informational','Browser_9','Browser_12',
                        'Browser_11','Region_8','Region_3','OperatingSystems_6','OperatingSystems_1','Month_June',
                        'Browser_8','Browser_7','Browser_3','Browser_13','Browser_10','VisitorType_Other','Region_8','Region_7',
                        'Region_5','OperatingSystems_5','Month_Aug'],axis=1)
X_Test = X_Test.drop(['Weekend_True','Region_2','Region_1','Month_Jul','Month_Feb','Informational','Browser_9','Browser_12',
                        'Browser_11','Region_8','Region_3','OperatingSystems_6','OperatingSystems_1','Month_June',
                        'Browser_8','Browser_7','Browser_3','Browser_13','Browser_10','VisitorType_Other','Region_8','Region_7',
                        'Region_5','OperatingSystems_5','Month_Aug'],axis=1)

In [240]:
X_Train = X_Train.drop(['OperatingSystems_4','OperatingSystems_7','OperatingSystems_8','Browser_5','Region_9'],axis=1)
X_Test = X_Test.drop(['OperatingSystems_4','OperatingSystems_7','OperatingSystems_8','Browser_5','Region_9'],axis=1)

print(X_Train.shape, X_Test.shape)

(8631, 23) (3699, 23)


In [241]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=33,sampling_strategy=0.43)
sm
X_Train_res, y_Train_res = sm.fit_sample(X_Train, y_Train)

In [242]:
pd.DataFrame(y_Train_res,columns=['Revenue']).Revenue.value_counts(normalize=True)

0    0.699358
1    0.300642
Name: Revenue, dtype: float64

In [243]:
print("Shape of x_train :", X_Train_res.shape) 
print("Shape of y_train :", y_Train_res.shape)

Shape of x_train : (10431, 23)
Shape of y_train : (10431,)


# Logistic Regression

In [244]:
from sklearn.linear_model import LogisticRegression
classifier_lg = LogisticRegression() 
classifier_lg.fit(X_Train_res,y_Train_res)
print(classifier_lg)

y_pred=classifier_lg.predict(X_Test)
warnings.filterwarnings("default")

# evaluating the model
from sklearn.metrics import roc_auc_score
print("Training Accuracy :", classifier_lg.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_lg.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

from sklearn.metrics import classification_report,confusion_matrix
print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_lg, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
Training Accuracy : 0.8373118588821781
Testing Accuracy : 0.889970262233036
ROC AUC Score : 0.7563645979859602
*******************************************************************************************
CONFUSION MATRIX
[[2970  157]
 [ 250  322]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      3127
           1       0.67      0.56      0.61       572

    accuracy                           0.89      3699
   macro avg       0.80      0.76      0.77      3699
weighted avg       0.88      0.89      0.8

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

CROSS VALIDATION METRICS
Mean Accuracy : 0.8370206285093562
Standard Deviation : 0.007907886855054837
Mean precision score : 0.8340379070843207
Standard Deviation precision score : 0.018153043101646445
Mean recall score : 0.5717343969394192
Standard Deviation recall score : 0.019410843305196736
Mean f1 score : 0.6782672621121499
Standard Deviation f1 score : 0.01734778814317086
Mean AUC ROC Score : 0.7613976680564958
Standard Deviation AUC ROC Score : 0.010852068020470807
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Logistic Regression Hyper-parameter Tuning

In [245]:
from sklearn.model_selection import GridSearchCV

base_classifier = LogisticRegression(penalty='l2')
params = [
    {
     'C' : [0.1, 0.01,1],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 0.01, 1],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
********************************************************

# Logistic Regression Hyper parameter tuned model

In [246]:
from sklearn.linear_model import LogisticRegression
classifier_lg = LogisticRegression(C=1,solver='newton-cg') 
classifier_lg.fit(X_Train_res,y_Train_res)
print(classifier_lg)

y_pred=classifier_lg.predict(X_Test)

# evaluating the model
from sklearn.metrics import roc_auc_score
print("Training Accuracy :", classifier_lg.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_lg.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

from sklearn.metrics import classification_report,confusion_matrix
print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_lg, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
Training Accuracy : 0.8373118588821781
Testing Accuracy : 0.8896999188969992
ROC AUC Score : 0.7554904721118344
*******************************************************************************************
CONFUSION MATRIX
[[2970  157]
 [ 251  321]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      3127
           1       0.67      0.56      0.61       572

    accuracy                           0.89      3699
   macro avg       0.80      0.76      0.77      3699
weighted avg       0.88      0.89     

# Naive Bayes

In [247]:
from sklearn.naive_bayes import GaussianNB
classifier_nb = GaussianNB() 
classifier_nb.fit(X_Train_res,y_Train_res)
print(classifier_nb)

y_pred=classifier_nb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_nb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_nb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_nb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GaussianNB(priors=None, var_smoothing=1e-09)
Training Accuracy : 0.7554405138529383
Testing Accuracy : 0.7510137875101379
ROC AUC Score : 0.7541707572887617
*******************************************************************************************
CONFUSION MATRIX
[[2344  783]
 [ 138  434]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.75      0.84      3127
           1       0.36      0.76      0.49       572

    accuracy                           0.75      3699
   macro avg       0.65      0.75      0.66      3699
weighted avg       0.85      0.75      0.78      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.754003801028742
Standard Deviation : 0.01207442839631649
Mean precision score : 0.5678261108438878
Standard Deviation precision sco

# KNN

In [248]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier() 
classifier_knn.fit(X_Train_res,y_Train_res)
print(classifier_knn)

y_pred=classifier_knn.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_knn.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_knn.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_knn, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Training Accuracy : 0.9184162592273032
Testing Accuracy : 0.8277912949445796
ROC AUC Score : 0.72315927596548
*******************************************************************************************
CONFUSION MATRIX
[[2735  392]
 [ 245  327]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.92      0.87      0.90      3127
           1       0.45      0.57      0.51       572

    accuracy                           0.83      3699
   macro avg       0.69      0.72      0.70      3699
weighted avg       0.85      0.83      0.84      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accu

# KNN Hyper-parameter tuning

In [249]:
base_classifier = KNeighborsClassifier()
params = [
    {
     'n_neighbors':[5,8,10,15],
        'leaf_size':[30,35,40],
        'weights':['uniform', 'distance'],
        'algorithm':['auto', 'ball_tree','kd_tree','brute']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'leaf_size': [30, 35, 40],
                          'n_neighbors': [5, 8, 10, 15],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
*******************************************************************************************
Best Hyper Parameters: {'algorithm': 'auto', 'leaf_size': 30, 'n

# KNN Hyper-parameter tuned model

In [250]:
classifier_knn = KNeighborsClassifier(algorithm='auto',leaf_size=30,n_neighbors=8,weights='distance') 
classifier_knn.fit(X_Train_res,y_Train_res)
print(classifier_knn)

y_pred=classifier_knn.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_knn.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_knn.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_knn, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='distance')
Training Accuracy : 1.0
Testing Accuracy : 0.8415788050824548
ROC AUC Score : 0.7313140569056782
*******************************************************************************************
CONFUSION MATRIX
[[2786  341]
 [ 245  327]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.92      0.89      0.90      3127
           1       0.49      0.57      0.53       572

    accuracy                           0.84      3699
   macro avg       0.70      0.73      0.72      3699
weighted avg       0.85      0.84      0.85      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.892

# Support Vector Machine

In [251]:
from sklearn.svm import SVC
classifier_svc = SVC() 
classifier_svc.fit(X_Train_res,y_Train_res)
print(classifier_svc)

y_pred=classifier_svc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_svc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_svc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_svc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
Training Accuracy : 0.871632633496309
Testing Accuracy : 0.8859151121924844
ROC AUC Score : 0.7789641203056619
*******************************************************************************************
CONFUSION MATRIX
[[2920  207]
 [ 215  357]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      3127
           1       0.63      0.62      0.63       572

    accuracy                           0.89      3699
   macro avg       0.78      0.78      0.78      3699
weighted avg       0.89      0.89      0.89      3699

**********************************************************************

C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

CROSS VALIDATION METRICS
Mean Accuracy : 0.8649225011576155
Standard Deviation : 0.011567517207773445
Mean precision score : 0.8234876674731353
Standard Deviation precision score : 0.019772929471758453
Mean recall score : 0.7009025050365275
Standard Deviation recall score : 0.02894657256424161
Mean f1 score : 0.7570662180002912
Standard Deviation f1 score : 0.023033978977662906
Mean AUC ROC Score : 0.8181686172701287
Standard Deviation AUC ROC Score : 0.016146887331432037
*******************************************************************************************


# Support Vector Machine hyper-parameter tuning

In [252]:
base_classifier = SVC()
params = [
    {
     'C': [0.1,1,5,7,8,9,10, 100, 1000],
        'gamma': [1,0.1,0.01,0.002,0.0005,0.001,0.0001],
        'kernel': ['rbf']
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 5, 7, 8, 9, 10, 100, 1000],
                          'gamma': [1, 0.1, 0.01, 0.002, 0.0005, 0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
*******************************************************************************************
Best Hyper Parameters: {'C': 7, 'gamma': 1, 'kernel': 'rbf'}
Best Accuracy Score: 0.9270443869235931
Best Estimators: SVC(C=7, cache_size=200, clas

# Support Vector Machine hyper-parameter tuned model

In [253]:
from sklearn.svm import SVC
classifier_svc = SVC(C=7,gamma=1,kernel='rbf') 
classifier_svc.fit(X_Train_res,y_Train_res)
print(classifier_svc)

y_pred=classifier_svc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_svc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_svc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_svc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

SVC(C=7, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
Training Accuracy : 0.9899338510209951
Testing Accuracy : 0.8572587185725872
ROC AUC Score : 0.6413104005045163
*******************************************************************************************
CONFUSION MATRIX
[[2983  144]
 [ 384  188]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3127
           1       0.57      0.33      0.42       572

    accuracy                           0.86      3699
   macro avg       0.73      0.64      0.67      3699
weighted avg       0.84      0.86      0.84      3699

***************************************************************************************

# AdaBoost

In [254]:
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

classifier_adb = AdaBoostClassifier() 
classifier_adb.fit(X_Train_res,y_Train_res)
print(classifier_adb)

y_pred=classifier_adb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_adb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_adb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_adb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
Training Accuracy : 0.8896558335730036
Testing Accuracy : 0.8786158421194917
ROC AUC Score : 0.7910741321358526
*******************************************************************************************
CONFUSION MATRIX
[[2870  257]
 [ 192  380]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      3127
           1       0.60      0.66      0.63       572

    accuracy                           0.88      3699
   macro avg       0.77      0.79      0.78      3699
weighted avg       0.88      0.88      0.88      3699

*******************************************************************************************
CROSS VALIDATION METRICS
Mean Accuracy : 0.8870867158680301
Standard Deviation : 0.0

# AdaBoost Hyper-parameter tuning

In [255]:
base_classifier = AdaBoostClassifier()
params = [
    {
     'n_estimators': [50,100,200,500],
          'learning_rate': [1,0.1,0.01,0.001],
          'base_estimator':[DecisionTreeClassifier(max_depth=1)]
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=None,
                                          learning_rate=1.0, n_estimators=50,
                                          random_state=None),
             iid='warn', n_jobs=-1,
             param_grid=[{'base_estimator': [DecisionTreeClassifier(class_weight=None,
                                                                    criterion='gini',
                                                                    max_depth=1,
                                                                    max_features=None,
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_impurity_split=None,
                           

# AdaBoost Hyper-parameter tuned model

In [256]:
classifier_adb = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'),learning_rate=1, n_estimators=500) 
classifier_adb.fit(X_Train_res,y_Train_res)
print(classifier_adb)

y_pred=classifier_adb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_adb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_adb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_adb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

# Gradient Boosting

In [257]:
from sklearn.ensemble import GradientBoostingClassifier
classifier_gb = GradientBoostingClassifier() 
classifier_gb.fit(X_Train_res,y_Train_res)
print(classifier_gb)

y_pred=classifier_gb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_gb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_gb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_gb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Training Accuracy : 0.9126641740964433
Testing Accuracy : 0.891051635577183
ROC AUC Score : 0.8005721093744759
*******************************************************************************************
CONFUSION MATRIX
[[2913  214]
 [ 189  383]]
*****************************************************************************

# Gradient Boosting Hyper-parameter tuning

In [258]:
base_classifier = GradientBoostingClassifier()
params = [
    {
    'n_estimators': [100,150,200,500],
          'learning_rate': [1,0.1,0.01,0.001],
        'max_depth': [1, 3, 5]
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort

# Gradient Boosting Hyper-parameter tuned model

In [259]:
from sklearn.ensemble import GradientBoostingClassifier
classifier_gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=200,max_depth=5) 
classifier_gb.fit(X_Train_res,y_Train_res)
print(classifier_gb)

y_pred=classifier_gb.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_gb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_gb.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_gb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
Training Accuracy : 0.9650081487872687
Testing Accuracy : 0.8915923222492566
ROC AUC Score : 0.7844646559069329
*******************************************************************************************
CONFUSION MATRIX
[[2938  189]
 [ 212  360]]
****************************************************************************

# Bagging Tree

In [260]:
from sklearn import tree
classifier_bagt = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)) 
classifier_bagt.fit(X_Train_res,y_Train_res)
print(classifier_bagt)

y_pred=classifier_bagt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_bagt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_bagt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_bagt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

# Bagging Tree hyper-parameter tuning

In [261]:
base_classifier = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
params = [
    {
    'n_estimators': [10,20,50],
        'max_features': [10,15,23],
        'max_samples':[10,15,20]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                                  

# Bagging Tree hyper-parameter tuned model

In [262]:
classifier_bagt = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1),
                                    max_features=23, max_samples=20, n_estimators=50) 
classifier_bagt.fit(X_Train_res,y_Train_res)
print(classifier_bagt)

y_pred=classifier_bagt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_bagt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_bagt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_bagt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

# Decision Tree

In [263]:
classifier_dt = DecisionTreeClassifier()
classifier_dt.fit(X_Train_res,y_Train_res)
print(classifier_dt)

y_pred=classifier_dt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_dt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_dt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_dt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Training Accuracy : 1.0
Testing Accuracy : 0.8553663152203298
ROC AUC Score : 0.7530391738098806
*******************************************************************************************
CONFUSION MATRIX
[[2818  309]
 [ 226  346]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3127
           1       0.53      0.60      0.56       572

    accuracy                           0.86      3699
   macro avg       0.73      0

# Decision Tree Hyper-parameter tuning

In [264]:
base_classifier = DecisionTreeClassifier()
params = [
    {
    'criterion':['gini','entropy'],
          'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123],
          'max_depth':[1,2,3,4,5,6]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [1, 2, 3, 4, 5, 6],
                  

# Decision Tree Hyper-parameter tuned model

In [265]:
classifier_dt = DecisionTreeClassifier(criterion='gini', max_depth=6,
                                       max_features='auto', min_samples_leaf=1, min_samples_split=5, random_state=123)
classifier_dt.fit(X_Train_res,y_Train_res)
print(classifier_dt)

y_pred=classifier_dt.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_dt.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_dt.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_dt, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')
Training Accuracy : 0.7616719394113699
Testing Accuracy : 0.8150851581508516
ROC AUC Score : 0.6156521364788075
*******************************************************************************************
CONFUSION MATRIX
[[2828  299]
 [ 385  187]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      3127
           1       0.38      0.33      0.35       572

    accuracy                           0.82      3699
   macro avg     

# RANDOM FOREST

In [266]:
classifier_rf = RandomForestClassifier(n_estimators=200)
classifier_rf.fit(X_Train_res,y_Train_res)
print(classifier_rf)

y_pred=classifier_rf.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_rf.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_rf.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_rf, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Training Accuracy : 1.0
Testing Accuracy : 0.8902406055690727
ROC AUC Score : 0.7858078522053578
*******************************************************************************************
CONFUSION MATRIX
[[2930  197]
 [ 209  363]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      3127
           1       0.65      0.63      0.64       572

# RF Hyper-parameter tuning

In [267]:
## Hyper Parameter Tuning
print('Parameters currently in use:\n')
print(classifier_rf.get_params())
print("\n")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 5)] ## play with start and stop

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 20,num = 5)] ## change 10,20 and 2
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,15]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,10]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

from sklearn.model_selection import RandomizedSearchCV

rf_random = RandomizedSearchCV(estimator = classifier_rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_Train_res, y_Train_res)
print("*******************************************************************************************")
print("Best Hyper Parameters:",rf_random.best_params_)
print("Best Accuracy Score:",rf_random.best_score_)
print("Best Estimators:",rf_random.best_estimator_)
print("*******************************************************************************************")

Parameters currently in use:

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)>


{'n_estimators': [50, 112, 175, 237, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 12, 15, 17, 20, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.1min finished


*******************************************************************************************
Best Hyper Parameters: {'n_estimators': 175, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 17, 'bootstrap': False}
Best Accuracy Score: 0.9121848336688716
Best Estimators: RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=17, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=175,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
*******************************************************************************************


# RF Hyper-parameter tuned model

In [269]:
classifier_rf = RandomForestClassifier(n_estimators=175,min_samples_split=5,min_samples_leaf=1,max_features='auto',max_depth=
                                       17,bootstrap=False)
classifier_rf.fit(X_Train_res,y_Train_res)
print(classifier_rf)

y_pred=classifier_rf.predict(X_Test)

# evaluating the model
print("Training Accuracy :", classifier_rf.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_rf.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_rf, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=17, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=175,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Training Accuracy : 0.9903173233630524
Testing Accuracy : 0.8905109489051095
ROC AUC Score : 0.7859677498708518
*******************************************************************************************
CONFUSION MATRIX
[[2931  196]
 [ 209  363]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      3127
           1       0.65      0.63      

# XGBOOST

In [271]:
from xgboost import XGBClassifier
classifier_xgb = XGBClassifier()
classifier_xgb.fit(X_Train_res,y_Train_res)
print(classifier_xgb)

y_pred=classifier_xgb.predict(X_Test.values)

# evaluating the model
print("Training Accuracy :", classifier_xgb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_xgb.score(X_Test.values, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_xgb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Training Accuracy : 0.9117054932413
Testing Accuracy : 0.8875371722087051
ROC AUC Score : 0.8034930371834752
*******************************************************************************************
CONFUSION MATRIX
[[2893  234]
 [ 182  390]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      3127
           1       0.62      0.68      0.65       572

    acc

# XGBOOST Hyper-parameter tuning

In [272]:
base_classifier = XGBClassifier()
params = [
    {
   'n_estimators': [100,150,200],
          'learning_rate': [1,0.1,0.01,0.015,0.025,0.05],
        'n_jobs':[-1]
        
    }
]

best_classifier = GridSearchCV(base_classifier, param_grid=params, n_jobs=-1,cv=10)

#Learning
best_classifier.fit(X_Train_res,y_Train_res)
print(best_classifier)
#The best hyper parameters set
print("*******************************************************************************************")
print("Best Hyper Parameters:",best_classifier.best_params_)
print("Best Accuracy Score:",best_classifier.best_score_)
print("Best Estimators:",best_classifier.best_estimator_)
print("*******************************************************************************************")

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'learning_rate': [1, 0.1, 0.01, 0.015, 0.025, 0.05],
                          'n_estimators': [100, 150, 200], 'n_jobs': [-1]}],
            

# XGBOOST Hyper-parameter tuned model

In [273]:
classifier_xgb = XGBClassifier(learning_rate=0.1, n_estimators=200, n_jobs=-1)
classifier_xgb.fit(X_Train_res,y_Train_res)
print(classifier_xgb)

y_pred=classifier_xgb.predict(X_Test.values)

# evaluating the model
print("Training Accuracy :", classifier_xgb.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", classifier_xgb.score(X_Test.values, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = classifier_xgb, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Training Accuracy : 0.9249352890422778
Testing Accuracy : 0.8918626655852934
ROC AUC Score : 0.7974806613277992
*******************************************************************************************
CONFUSION MATRIX
[[2921  206]
 [ 194  378]]
*******************************************************************************************
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3127
           1       0.65      0.66      0.65       572

   

# VOTING CLASSIFIER

In [277]:
lr_vc = LogisticRegression(C=1,solver='newton-cg')
svm_vc=SVC(C=7,gamma=1,kernel='rbf')
knn_vc=KNeighborsClassifier(algorithm='auto',leaf_size=30,n_neighbors=8,weights='distance')
nb_vc=GaussianNB()
dt_vc=DecisionTreeClassifier(criterion='gini', max_depth=6,
                                       max_features='auto', min_samples_leaf=1, min_samples_split=5, random_state=123)
rf_vc=RandomForestClassifier(n_estimators=175,min_samples_split=5,min_samples_leaf=1,max_features='auto',max_depth=
                                       17,bootstrap=False)

evc = VotingClassifier(estimators= [('dt',dt_vc),('svm',svm_vc),('knn',knn_vc),('nb',nb_vc),('rf',rf_vc),('lr',lr_vc)], 
                       voting = 'hard')
evc.fit(X_Train_res,y_Train_res)
print(evc.fit)


y_pred=evc.predict(X_Test)

# evaluating the model
print("Training Accuracy :", evc.score(X_Train_res, y_Train_res))
print("Testing Accuracy :", evc.score(X_Test, y_Test))
print("ROC AUC Score :", roc_auc_score(y_Test, y_pred))
print("*******************************************************************************************")

print("CONFUSION MATRIX")
print(confusion_matrix(y_Test,y_pred))
print("*******************************************************************************************")
print("CLASSIFICATION REPORT")
print(classification_report(y_Test,y_pred))
print("*******************************************************************************************")

# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = evc, X = X_Train_res, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

<bound method VotingClassifier.fit of VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=6,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=5,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                                     random_state=123,
                                          

# STACKING

In [282]:
## Algorithm 1: xgboost
pred_val_xgb=classifier_xgb.predict(X_Train_res)
test_pred_xgb=classifier_xgb.predict(X_Test.values)

## Algorithm 2: Random Forest
pred_val_rf=classifier_rf.predict(X_Train_res)
test_pred_rf=classifier_rf.predict(X_Test)

## Algorithm 3: Decision Tree
pred_val_dt=classifier_dt.predict(X_Train_res)
test_pred_dt=classifier_dt.predict(X_Test)

stacked_predictions=np.column_stack((pred_val_rf,pred_val_xgb,pred_val_dt)) ## Prediction by the algorithms on training data
#stacked_predictions[0:10]
stacked_test_predictions=np.column_stack((test_pred_rf,test_pred_xgb,test_pred_dt)) ## Prediction by the algorithms on testing data
#stacked_test_predictions[0:10]

## Building Meta Model
lr_stack = LogisticRegression()
lr_stack.fit(stacked_predictions,y_Train_res) ## stacked_predictions=X_train

y_pred_stack=lr_stack.predict(stacked_test_predictions) ## stacked_test_predictions=X_test
y_pred_stack


# evaluating the model
cm = confusion_matrix(y_Test, y_pred_stack)
print(cm)
print(classification_report(y_Test, y_pred_stack)) 

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = lr_stack, X = stacked_predictions, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

[[2931  196]
 [ 209  363]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      3127
           1       0.65      0.63      0.64       572

    accuracy                           0.89      3699
   macro avg       0.79      0.79      0.79      3699
weighted avg       0.89      0.89      0.89      3699

CROSS VALIDATION METRICS
Mean Accuracy : 0.9903218098408552
Standard Deviation : 0.006077431918858364
Mean precision score : 0.9890598329005325
Standard Deviation precision score : 0.00455828855967136
Mean recall score : 0.9786573329806068
Standard Deviation recall score : 0.01973382444762793
Mean f1 score : 0.9837268099408378
Standard Deviation f1 score : 0.010335677781670879
Mean AUC ROC Score : 0.9869983021330491
Standard Deviation AUC ROC Score : 0.009905448900745604
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

# BLENDING

In [283]:
## Algorithm 1: xgboost
pred_val_xgb=classifier_xgb.predict(X_Train_res)
test_pred_xgb=classifier_xgb.predict(X_Test.values)

## Algorithm 2: Random Forest
pred_val_rf=classifier_rf.predict(X_Train_res)
test_pred_rf=classifier_rf.predict(X_Test)

## Algorithm 3: Decision Tree
pred_val_dt=classifier_dt.predict(X_Train_res)
test_pred_dt=classifier_dt.predict(X_Test)

stacked_predictions=np.column_stack((pred_val_rf,pred_val_xgb,pred_val_dt)) ## Prediction by the algorithms on training data
#stacked_predictions[0:10]
stacked_test_predictions=np.column_stack((test_pred_rf,test_pred_xgb,test_pred_dt)) ## Prediction by the algorithms on testing data
#stacked_test_predictions[0:10]

stacked_predictions=pd.DataFrame(stacked_predictions)
stacked_test_predictions=pd.DataFrame(stacked_test_predictions)


## Building Meta Model
lr_stack = LogisticRegression()
lr_stack.fit(stacked_predictions,y_Train_res) ## stacked_predictions=X_train

y_pred_stack=lr_stack.predict(stacked_test_predictions) ## stacked_test_predictions=X_test
y_pred_stack


# evaluating the model
cm = confusion_matrix(y_Test, y_pred_stack)
print(cm)
print(classification_report(y_Test, y_pred_stack)) 

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'AUC ROC score' : make_scorer(roc_auc_score)}
cvs = cross_validate(estimator = lr_stack, X = stacked_predictions, y = y_Train_res, cv = 10,scoring=scoring)
#print(cvs)
print("CROSS VALIDATION METRICS")
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Standard Deviation f1 score :", np.std(cvs['test_f1_score']))
print("Mean AUC ROC Score :", np.mean(cvs['test_AUC ROC score']))
print("Standard Deviation AUC ROC Score :", np.std(cvs['test_AUC ROC score']))
print("*******************************************************************************************")

[[2931  196]
 [ 209  363]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      3127
           1       0.65      0.63      0.64       572

    accuracy                           0.89      3699
   macro avg       0.79      0.79      0.79      3699
weighted avg       0.89      0.89      0.89      3699

CROSS VALIDATION METRICS
Mean Accuracy : 0.9903218098408552
Standard Deviation : 0.006077431918858364
Mean precision score : 0.9890598329005325
Standard Deviation precision score : 0.00455828855967136
Mean recall score : 0.9786573329806068
Standard Deviation recall score : 0.01973382444762793
Mean f1 score : 0.9837268099408378
Standard Deviation f1 score : 0.010335677781670879
Mean AUC ROC Score : 0.9869983021330491
Standard Deviation AUC ROC Score : 0.009905448900745604
*******************************************************************************************


C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neera\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t